In [50]:
import urllib.request as ur
import re
import pandas as pd


In [81]:
samples = ['J000092712', 'J000106560', 'TM00016', 'TM00065', 'TM00089', 'TM00090', 'TM00090', 'TM00096', 'TM00107', 'TM00137', 'TM00170', 'TM00226', 'TM00231', 'TM00256', 'TM00279', 'TM00314', 'TM00335', 'TM00999', 'TM01571']
#samples = ['J000092712', 'TM00090']

In [82]:
jax_strna = pd.DataFrame({'Gene': []}).set_index('Gene')
jax_rna = pd.DataFrame({'Gene': []}).set_index('Gene')

for sample in samples:
    page = ur.urlopen(f'http://tumor.informatics.jax.org/mtbwi/pdxDetails.do?modelID={sample}').read().split(b'\r\n')
    for line in page:
        data,count = re.subn(b'\s*expressionData = google.visualization.arrayToDataTable\((.*)\);\s*',b'\g<1>',line)
        if count:
            data = re.sub(b'true',b'True',data)
            data = re.sub(b'false',b'False',data)
            data = re.sub(b"{role:'certainty'}",b'None',data)
            data = re.sub(b'null',b'None',data)
            data = eval(data)
            expr_data = pd.DataFrame(data[1:],columns=data[0]).set_index('Gene')
            counts = expr_data.filter(like='stRNA_Seq')
            if not counts.columns.empty:
                counts.columns=[sample]
                jax_strna = pd.concat([jax_strna, counts],axis = 1)
            else:
                counts = expr_data.filter(like=' RNA_Seq')
                counts.columns=[sample]
                jax_rna = pd.concat([jax_rna,counts],axis = 1) 

TypeError: Index(...) must be called with a collection of some kind, 'Gene' was passed

In [83]:
jax_rna

,J000092712,TM00016,TM00089,TM00096,TM00137,TM00226,TM00256,TM00999,TM01571
Gene,,,,,,,,,
A1CF,-0.36,0.39,-0.58,-0.40,1.10,-0.23,0.09,-0.52,-0.19
ABL1,1.32,-0.26,-0.55,-1.78,0.09,-0.21,0.64,0.81,1.59
ACVR1B,-1.63,-0.60,-1.37,0.64,0.76,1.43,0.62,-1.44,-1.05
ADAR,0.96,0.58,0.87,0.63,-2.45,0.50,0.46,1.15,-1.79
ADARB1,2.15,0.41,0.58,0.65,-0.55,0.49,-0.63,1.95,0.56
...,...,...,...,...,...,...,...,...,...
XRCC3,0.27,1.29,-1.38,-0.09,0.00,-0.99,-0.73,-0.41,-0.41
YAP1,0.25,0.25,-0.68,0.41,0.31,0.52,0.18,0.12,0.20
YES1,0.53,0.81,0.79,0.82,-0.91,0.79,0.02,1.30,0.44


In [84]:
jax_strna

,J000106560,TM00065,TM00090,TM00090,TM00107,TM00170,TM00231,TM00279,TM00314,TM00335
Gene,,,,,,,,,,
A1CF,0.46,0.37,0.48,0.48,0.40,1.71,0.66,0.73,1.70,0.32
ABL1,1.48,1.35,0.42,0.42,0.75,-0.05,-1.26,1.41,-0.18,-0.90
ACVR1B,-0.63,-1.23,-0.60,-0.60,-1.14,1.47,-0.04,-0.64,1.18,1.06
ADAR,0.79,0.02,1.05,1.05,1.00,0.11,-0.30,-2.20,-0.08,0.93
ADARB1,0.11,1.16,0.08,0.08,0.03,-0.44,0.18,1.67,-0.94,0.85
...,...,...,...,...,...,...,...,...,...,...
XRCC3,-0.92,-1.40,0.47,0.47,-0.23,0.04,0.63,-0.40,-1.88,-0.11
YAP1,0.55,0.47,-0.09,-0.09,0.29,0.52,0.51,0.51,0.40,0.51
YES1,0.07,0.48,0.31,0.31,0.63,-0.40,0.72,0.33,0.24,-1.13
